***GENERATED CODE FOR regressionmodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regressionmodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	cleandata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-3e1740f5-8f37-4076-a4c8-d718e62b8e1a-c000.csv', 'filename': '1707886543CleanData.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Profit Dataset analysis/CleanData.csv', 'viewFileName': 'CleanData.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressionmodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	regressionmodelautofe = TransformationMain.run(cleandata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Postal Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8994", "mean": "55200.86", "stddev": "32076.22", "min": "1040", "max": "99301", "missing": "0"}, "updatedLabel": "Postal Code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profit Ratio", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "0.16", "stddev": "5.81", "min": "-292.0", "max": "157.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profit Ratio"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "3.7", "stddev": "2.62", "min": "-38.0", "max": "106.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "227.36", "stddev": "625.24", "min": "-1.0", "max": "22638.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales Per Customer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "229.26", "stddev": "625.03", "min": "0.0", "max": "22638.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales Per Customer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Number of Customer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "3.17", "stddev": "25.09", "min": "1.0", "max": "1082.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Number of Customer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8994", "mean": "0.58", "stddev": "0.77", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Category_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "City_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "65.98", "stddev": "102.56", "min": "0.0", "max": "530.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "City_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Country_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8994", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Country_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "283.41", "stddev": "210.98", "min": "0.0", "max": "792.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer Name_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "283.67", "stddev": "211.23", "min": "0.0", "max": "792.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer Name_stringindex..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order Date_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "363.19", "stddev": "297.06", "min": "0.0", "max": "1236.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order Date_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "1660.69", "stddev": "1412.59", "min": "0.0", "max": "5008.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "673.06", "stddev": "488.79", "min": "0.0", "max": "1860.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Product ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product Name_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "659.57", "stddev": "488.19", "min": "0.0", "max": "1846.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Product Name_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "1.67", "stddev": "3.23", "min": "0.0", "max": "42.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Row ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "4709.08", "stddev": "2872.3", "min": "0.0", "max": "9701.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Row ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "4.59", "stddev": "27.71", "min": "0.0", "max": "288.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Date_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "395.52", "stddev": "329.68", "min": "0.0", "max": "1346.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Date_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "4.07", "stddev": "23.51", "min": "0.0", "max": "256.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "State_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "8.42", "stddev": "10.36", "min": "0.0", "max": "63.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "State_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sub-Category_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "5.36", "stddev": "5.7", "min": "0.0", "max": "66.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sub-Category_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Top N Customer Label_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "0.16", "stddev": "0.86", "min": "0.0", "max": "19.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Top N Customer Label_stri..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Discount_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "1.28", "stddev": "2.19", "min": "0.0", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Discount_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Number of Records_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "0.06", "stddev": "0.47", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Number of Records_stringi..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profit_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8994", "mean": "2643.41", "stddev": "2225.6", "min": "0.0", "max": "7081.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profit_stringindexer"}]}))

	#transformationPostExecutionHook(regressionmodelautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressionmodelHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regressionmodelautofe, ["Postal Code", "Profit Ratio", "Quantity", "Sales", "Sales Per Customer", "Total Number of Customer", "Category_stringindexer", "City_stringindexer", "Country_stringindexer", "Customer ID_stringindexer", "Customer Name_stringindexer", "Order Date_stringindexer", "Order ID_stringindexer", "Product ID_stringindexer", "Product Name_stringindexer", "Region_stringindexer", "Row ID_stringindexer", "Segment_stringindexer", "Ship Date_stringindexer", "Ship Mode_stringindexer", "State_stringindexer", "Sub-Category_stringindexer", "Top N Customer Label_stringindexer", "Discount_stringindexer", "Number of Records_stringindexer"], "Profit_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

